In [1]:
!pip install statsmodels

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


We filter for companies that have been public for at least 60 months.

We begin our pre-processing.

In [3]:
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/ecma_33130_raw.csv')
raw_data.datadate = pd.to_datetime(raw_data.datadate)
raw_data.ipodate = pd.to_datetime(raw_data.ipodate)
last_date = raw_data.groupby('gvkey').last()['datadate']
first_date = raw_data.groupby('gvkey').first()['ipodate']
first_date.where(pd.notnull,
                 raw_data.groupby('gvkey').first()['datadate'],
                 inplace=True)
comp_to_keep = first_date + pd.DateOffset(months = 60) < last_date
df = raw_data[raw_data['gvkey'].isin(comp_to_keep[comp_to_keep].index)]
df = df.reset_index(drop=True)
# Momentum Features
df = df.sort_values(['gvkey', 'datadate'])
# trailing returns
for n in [3, 9, 12, 18]:
    df[f'ret_{n}m'] = df.groupby('gvkey')['prccq'].pct_change(n, fill_method=None)
    df[f'rel:ret_{n}m'] = df.groupby('gvkey')[f'ret_{n}m'].rank(pct=True)
# book to market =  shareholder equity / (price close * common shares outstanding)
df['bkmktq'] = df['seqq'] / (df['prccq'] * df['cshoq'])
df['rel:bkmktq'] = df.groupby('datadate')['bkmktq'].rank(pct=True)
# Enterprise Value
df['evq'] = (
    df['prccq'] * df['cshoq']  # Market capitalization
    + df['dlttq']              # Long-term debt
    + df['dlcq']               # Debt in current liabilities
    - df['cheq']               # Cash and short-term investments
)
# Earnings yield
df['eyq'] = df['oiadpq'] / df['evq']
df['rel:eyq'] = df.groupby('datadate')['eyq'].rank(pct=True)
# NaN Policy
df.replace([np.inf, -np.inf], np.nan, inplace=True)
nan_cols = [
    f'is_nan:{col}' for col in df.columns
    if col not in ['gvkey', 'datadate']
]
cols_to_nan = [col for col in df.columns if col not in ['gvkey', 'datadate']]
nan_df = df[cols_to_nan].isna().astype(int)
nan_df.columns = nan_cols
df = pd.concat([df, nan_df], axis=1)
df[cols_to_nan] = df.groupby(['gvkey', 'datadate'])[cols_to_nan].ffill()
df[cols_to_nan] = df[cols_to_nan].fillna(0)
# transform
numrcl_cols = [
    col for col in df.columns
    if col not in ['gvkey', 'datadate', 'fyearq', 'fqtr', 'exchg', 'ipodate']
    and col[:3] != 'rel'
    and col[:6] != 'is_nan'
    and pd.api.types.is_numeric_dtype(df[col])
]
min_val = df[numrcl_cols].min(axis=None)
df[numrcl_cols] += np.abs(min_val)
import numba
norm_factor = (
    df.groupby(['gvkey', 'datadate'])[numrcl_cols]
      .expanding(min_periods= 1, method = 'table')
      .mean(numeric_only = True, engine = 'numba', engine_kwargs={'parallel':True})
      .reset_index(drop = True)
)
df[numrcl_cols] /= norm_factor[numrcl_cols]
df[numrcl_cols] = np.log1p(df[numrcl_cols])

In [4]:
# deseasonalization and detrend
from statsmodels.tsa.seasonal import STL
cols_to_deszn = [
    col for col in df.columns
    if (
        col not in ['gvkey', 'datadate', 'fyearq', 'fqtr', 'exchg', 'ipodate']
        and col[:6] != 'is_nan'
        and df[col].dtype in [np.float64, np.int64]
        and df[col].nunique() > 4
    )
]

In [5]:
def deszn(df, cols_to_deszn):
  szn_cols = {col:f'szn:{col}' for col in cols_to_deszn}
  szns = df.groupby('gvkey')[cols_to_deszn].transform(
    lambda x: STL(
        x,
        period = 4,
        seasonal = 9,
        robust = True
    ).fit().seasonal,
  )
  szns = szns.rename(columns=szn_cols)
  df = pd.concat([df, szns], axis=1)
  return df

In [6]:
df.loc[34842, 'evq'] = df.loc[34841, 'evq']
df.loc[34842, 'is_nan:evq'] = 1

In [7]:
df = deszn(df, cols_to_deszn)

We now massage our data into the pytorch forecasting dataset.

In [9]:
df['val_trap'] = df.groupby('gvkey')['ret_12m'].rank(pct=True) <= 0.1

In [8]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/ecma_33130_data.csv')

In [10]:
# these can't be categorized, since they will not appear in the training
# and are reflected in the time
variables_to_drop = [
    'fyearq', 'datacqtr', 'datafqtr'
]
static_categoricals = [
  'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'costat', 'loc',
]
time_varying_known_categoricals=[
  'gvkey', 'fqtr'
]
time_varying_known_reals=[
  'ipodate'
]
time_varying_unknown_categoricals=[
    col for col in df.columns
    if col[:6] == 'is_nan'
] + ['val_trap']
time_varying_unknown_reals=[
  col for col in df.columns
  if col[:3] == 'szn'
] + [
  col for col in df.columns
  if col[:3] == 'rel'
] + [
  col for col in df.columns
  if col[:3] == 'ret'
] + [
  'eyq', 'evq', 'bkmktq', 'prclq', 'prccq', 'cdvcy', 'capxy', 'seqq',
  'saleq', 'rectq', 'pstkq', 'ppentq', 'oiadpq', 'niq', 'ltq', 'lctq',
  'intanq', 'dlttq', 'dlcq', 'cogsq', 'cshoq', 'cheq', 'atq', 'apq', 'actq'
]
# variable_groups = {
#     'is_nan': [col for col in df.columns if col[:6] == 'is_nan'],
# }
#need to categorify categorical variables
for col in static_categoricals:
  df[col] = df[col].astype('str')
for col in time_varying_known_categoricals:
  df[col] = df[col].astype('str')
for col in time_varying_unknown_categoricals:
  df[col] = df[col].astype('str')
df['ipodate'] = pd.factorize(df['ipodate'])[0]
df = df.drop(labels=variables_to_drop, axis = 1)
df['time'] = pd.factorize(df['datadate'])[0]
train_df = df[pd.to_datetime(df['datadate']) <= pd.Timestamp(2009, 12, 31)].drop(labels='datadate', axis=1)
test_df = df[pd.to_datetime(df['datadate']) >= pd.Timestamp(2010, 1, 1)].drop(labels='datadate', axis=1)
df = df.drop(labels='datadate', axis=1)

In [11]:
!pip install -q pytorch-forecasting
from pytorch_forecasting import TimeSeriesDataSet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
data = TimeSeriesDataSet(
    df,
    time_idx='time',
    target='val_trap',
    group_ids=['gvkey'],
    max_encoder_length=20,
    min_encoder_length=20,
    max_prediction_length=20,
    static_categoricals = static_categoricals,
    time_varying_known_categoricals=time_varying_known_categoricals,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_categoricals=time_varying_unknown_categoricals,
    time_varying_unknown_reals=time_varying_unknown_reals,
    add_relative_time_idx=True,
    allow_missing_timesteps=True
)
train = TimeSeriesDataSet.from_dataset(
    data,
    train_df,
)
test = TimeSeriesDataSet.from_dataset(
    data,
    test_df,
    predict=True,
    stop_randomization=True,
)

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 3937 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__gvkey': '1000'}, {'__group_id__gvkey': '10001'}, {'__group_id__gvkey': '10012'}, {'__group_id__gvkey': '10014'}, {'__group_id__gvkey': '10017'}, {'__group_id__gvkey': '10036'}, {'__group_id__gvkey': '10039'}, {'__group_id__gvkey': '10066'}, {'__group_id__gvkey': '10075'}, {'__group_id__gvkey': '10076'}]
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 3936 series/groups which therefore are not present in the dataset index. This means no predictions can be

In [ ]:
# save datasets
train.save('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/train.pt')
test.save('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/test.pt')